# Hyperparameter Tuning ANN

Guidelines to finding the optimal number of Hidden Layers & Hidden Neurons in ANN

- Start simple
- Grid Search / Random Search
- Cross Validation
- Heuristics & Rules of Thumb
  - Number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer
  - Start with 1 - 2 Hidden Layers

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline

# from keras.wrappers import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

# Import Dataset

In [ ]:
df0 = pd.read_csv('Customer-Churn-Records.csv')
df = df0.copy() 

In [ ]:
df.info()

In [ ]:
df

---

## Data Cleaning

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
df.columns.tolist()

In [ ]:
df.columns = df.columns.str.replace(' ', '_')

In [ ]:
df.columns.tolist()

---

## Feature Engineering

In [ ]:
df = df.drop(['rownumber','customerid','surname','card_type'], axis=1)

In [ ]:
df

---

## Data Encoding

In [ ]:
lbl_encoder = LabelEncoder()

In [ ]:
df['gender'] = lbl_encoder.fit_transform(df['gender'])

In [ ]:
df